
# Preprocesamiento

In [2]:
import numpy as np
import pandas as pd


In [3]:
# Se leen los datos del archivo csv
data = pd.read_csv("MovieGenre.csv", encoding ="ISO-8859-1")
data.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [4]:
data.isnull().sum()

imdbId          0
Imdb Link       0
Title           0
IMDB Score     48
Genre         145
Poster        724
dtype: int64

# WebScraping

In [6]:
# Se obtienen los resumenes de las peliculas
import requests
from bs4 import BeautifulSoup
import csv

def get_synopsis(link):
    base_link = "http://www.imdb.com"
    page = requests.get(base_link + link)
    soup = BeautifulSoup(page.content, 'html.parser')
    text = soup.find(id="plot-synopsis-content").get_text()
    return text

def get_text_data(movie_links, filename):    
    with open(filename, "w") as csvfile:        
        field_names = ["Imdb Link", "Synopsis Link", "Synopsis"]
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        writer.writeheader()

        for link in movie_links:
            csv_fields = {}
            csv_fields['Imdb Link'] = link
            page = requests.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')
            storyline_block = soup.find(id="titleStoryLine")
            if storyline_block is None:
                print(link)
                continue          
                
            see_more_block = storyline_block.find("span",{'class':"see-more inline"})        
            sm_link_blocks = see_more_block.find_all("a")
            for sm_link_block in sm_link_blocks:
                if sm_link_block.get_text() == "Plot Synopsis":
                    sm_link = sm_link_block['href']
                    csv_fields['Synopsis Link'] = sm_link
                    csv_fields['Synopsis'] = get_synopsis(sm_link)
                    break

            writer.writerow(csv_fields)
            
def divide_list(n_chunks, list_to_divide):
    # TODO
    # Move to utils file
    chunk_size = len(list_to_divide)//n_chunks + 1
    chunks = [list_to_divide[i:i+chunk_size] for i in range(0, len(list_to_divide), chunk_size)]
    
    return chunks
    
import multiprocessing
n_jobs = 2
chunks = divide_list(n_jobs, data['Imdb Link'])


jobs = []
for i, chunk in enumerate(chunks):    
    filename = "Movie_Synopsis_" + str(i) + ".csv"
    p = multiprocessing.Process(target=get_text_data,
                                args=(chunk, filename))
    jobs.append(p)
    p.start()
    
for job in jobs:
    job.join()

http://www.imdb.com/title/tt1623288
http://www.imdb.com/title/tt114709
http://www.imdb.com/title/tt113497
http://www.imdb.com/title/tt2023453
http://www.imdb.com/title/tt113228
http://www.imdb.com/title/tt1437833
http://www.imdb.com/title/tt114885
http://www.imdb.com/title/tt256106
http://www.imdb.com/title/tt113041
http://www.imdb.com/title/tt37154
http://www.imdb.com/title/tt113277
http://www.imdb.com/title/tt36036
http://www.imdb.com/title/tt312450
http://www.imdb.com/title/tt114319
http://www.imdb.com/title/tt2066040
http://www.imdb.com/title/tt112302
http://www.imdb.com/title/tt114576
http://www.imdb.com/title/tt454155
http://www.imdb.com/title/tt113189
http://www.imdb.com/title/tt1542928
http://www.imdb.com/title/tt112346
http://www.imdb.com/title/tt2025526
http://www.imdb.com/title/tt112896
http://www.imdb.com/title/tt1728196


Process Process-22:
Process Process-21:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-9a1cabcad25f>", line 22, in get_text_data
    page = requests.get(link)
  File "/opt/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/opt/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/opt/anaconda3/lib/python3.6/site-packages/requests/sessions.py", li